# Requirements

In [1]:
import os
import hickle as hkl
import h5py
import numpy as np
import requests
import time
import concurrent.futures
import pandas as pd

---
# save links to a file

In [4]:
df = pd.read_csv('Datasets/Air Conditioners.csv')
img_urls = df['image'].values
np.savetxt('images/img_urls.txt', img_urls, fmt='%s')

In [53]:
with open('images/img_urls.txt', 'r') as url_list:
    img_urls = [str(line) for line in url_list.readlines() if line[-2] != 'f']

---
# Store images in the dataset

In [52]:
def store_images(images, labels):
    with h5py.File('images/images.h5', 'w') as hf:
        hf.create_dataset('images', np.shape(images), h5py.h5t.STD_U8BE, data=images)
        hf.create_dataset('labels', np.shape(labels), h5py.h5t.STD_U8BE, data=labels)

# Download images

In [54]:
def download_image(img_url):
    img_bytes = requests.get(img_url).content
    img_name = img_url.split('/')[3]
    img_name = f'{img_name}.jpg'
    with open(img_name, 'wb') as img_file:
        img_file.write(img_bytes)
        print(f'{img_name} was downloaded...')

In [55]:
with concurrent.futures.ThreadPoolExecutor() as executor:
    for url in img_urls:
        executor.submit(download_image, url)

images.jpg was downloaded...images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...

images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was downloaded...
images.jpg was